In [3]:

import pandas as pd
import numpy as np
import time
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from collections import namedtuple

In [4]:
Volume = {}
data = pd.read_csv('SPY.csv')
test_data = data.Date.str.contains('4/28/2020')
df = data[test_data]

trade = namedtuple('trade', 'Side Entry Exit Profit')

In [19]:
def format_date(t):
    if len(t)<11:
        t = t+' 12:00' #somehow the data is missing the 12pm timem stamp
    date,time = t.split(' ')
    month,day,year = date.split('/')
    hr,sec = time.split(':')

    return dt.datetime(int(year),int(month),int(day),int(hr),int(sec))
        
def calc_voc(data,factor):
    volume ={}
    
    for indx, row in data.iterrows():
        high_price = round(row['High']*100)
        low_price = round(row['Low']*100)
        num = high_price-low_price+1
        avg_vol = row['Volume']/num
        for i in range(low_price,high_price+1):
            if i in volume:
                volume[i] = volume[i]+avg_vol
            else:
                volume[i] = avg_vol
            
        
    return volume
def get_high_volume(data):
    return max([x for x in data.items()], key = lambda x :x[1])[0]/100
def poc_tester(data, TP, SL):
    current_day = None
    volume = {}
    entry_price = 0
    close_price = 0
    side = -1#1 indicates buy and 0 indicates sell
    win_list = [] 
    lose_list = []
    VOC_price = 0
    last_price = 0
    for index, row in data.iterrows():
        date = format_date(row['Date'])
        if date.hour>15:
            continue

        if date.hour<9: 
            continue

        if date.hour == 9 and date.minute<30:
            continue
            

        
        if current_day == None or date.day != current_day: # new data reset everyhing
            #new_day
            if entry_price != 0 and side != -1: # close at end of day
                if side == 1:# buy
                    if entry_price<last_price:
                        close_price = last_price
                        win_list.append(trade(side,entry_price,close_price,(close_price-entry_price)/entry_price))

                    elif entry_price>=last_price: 
                        close_price = last_price
                        lose_list.append(trade(side,entry_price,close_price,(entry_price-close_price)/entry_price))
                        
                elif side ==0: # sell
                    if last_price < entry_price: #price lower than sell price:
                        close_price = last_price
                        win_list.append(trade(side,entry_price,close_price,(entry_price-close_price)/entry_price))
                    elif last_price >= entry_price:
                        close_price = last_price
                        lose_list.append(trade(side,entry_price,close_price,(close_price-entry_price)/entry_price))

            current_day = date.day
            side = -1
            entry_price = 0
            close_price = 0

            if len(volume) == 0:
                VOC_price = 0
            else:
                VOC_price = get_high_volume(volume)

            volume = {}
            
        #calculate VOC
        high_price = round(row['High']*100)
        low_price = round(row['Low']*100)
        num = high_price-low_price+1
        avg_vol = row['Volume']/num
        for i in range(low_price,high_price+1):
            if i in volume:
                volume[i] = volume[i]+avg_vol
            else:
                volume[i] = avg_vol
        #end
        if VOC_price ==0:
            continue
            
        last_price = row['Open']
        if side == -1 and entry_price == 0:
            if row['Open'] > VOC_price:
                side = 1
            else:
                side = 0
        if side == 1 and entry_price == 0:
            if row['Low'] < VOC_price:

                entry_price = VOC_price
                continue
        
        elif side == 0 and entry_price == 0:

            if row['High']>VOC_price:

                entry_price = VOC_price
                continue
        if entry_price != 0:
            if side == 1:# buy
                
                if row['High']>entry_price*(1+TP):
                    #add to wining trade
                    close_price = entry_price*(1+TP)
                    win_list.append(trade(side,entry_price,entry_price*(1+TP),(close_price-entry_price)/entry_price))

                    side =-1
                    entry_price =0 

                elif row['Low']<entry_price*(1-SL): #price less than SL
                    #add to losing trad
                    close_price = entry_price*(1-SL)
                    lose_list.append(trade(side,entry_price,entry_price*(1-SL),(entry_price-close_price)/entry_price))

                    side = -1
                    entry_price = 0

                else:
                    continue
            elif side ==0: # sell
                if row['Low']<entry_price*(1-TP): #price lower than TP:
                    #add to wining trade
                    close_price = entry_price*(1-TP)

                    win_list.append(trade(side,entry_price,entry_price*(1-TP),(entry_price-close_price)/entry_price))
                    side =-1
                    entry_price =0 
              
                elif row['High']>entry_price*(1+SL): #price less than SL
                    #add to losing trade

                    close_price = entry_price*(1+SL)
                    lose_list.append(trade(side,entry_price,entry_price*(1+SL),(close_price-entry_price)/entry_price))
                    side = -1
                    entry_price = 0

                else:
                    continue
    return win_list,lose_list

In [20]:

#test case should have no trades at all
print("test_case 1")
day1 = '4/24/2020'
day2 ='4/27/2020'
data = pd.read_csv('SPY.csv')
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data = pd.concat([data_day1,data_day2], axis=0)
data
print(poc_tester(data,0.002,0.002) == ([],[]))

#test case should 2 trades
print("test_case2")
day1 = '4/27/2020'
day2 ='4/28/2020'
data = pd.read_csv('SPY.csv')

test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data = pd.concat([data_day1,data_day2], axis=0)
print(poc_tester(data,0.005,0.005))
print(poc_tester(data,0.005,0.005) ==([trade(Side=0, Entry=287.04, Exit=285.6048, Profit=0.0050000000000000305)], [trade(Side=1, Entry=287.04, Exit=285.6048, Profit=0.0050000000000000305)]))

test_case 1
True
test_case2
([trade(Side=0, Entry=287.04, Exit=285.6048, Profit=0.0050000000000000305)], [trade(Side=1, Entry=287.04, Exit=285.6048, Profit=0.0050000000000000305)])
True


In [35]:
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for name in files:
    print(name)
    data = pd.read_csv(name)
    win_list,lose_list  = poc_tester(data,0.004,0.002)
    total_win = 0
    total_lose =0 
    for i in win_list:
        assert(i.Profit>=0)
        total_win +=i.Profit
    for i in lose_list:
        assert(i.Profit>=0)
        total_lose +=i.Profit

    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))

TLT.csv
total of 39 wining trades at a 0.13866578800786983 gain
total of 67 losing trades at a 0.12820516864225195 loss
SPY.csv
total of 59 wining trades at a 0.20739805609573259 gain
total of 134 losing trades at a 0.2511120115349213 loss
APPL.CSV
total of 114 wining trades at a 0.43540127054421934 gain
total of 186 losing trades at a 0.3674669601270343 loss
IWM.csv
total of 77 wining trades at a 0.269778210503463 gain
total of 137 losing trades at a 0.27254203806311766 loss
AMZN.csv
total of 80 wining trades at a 0.30083626165683297 gain
total of 194 losing trades at a 0.3852011753587926 loss


In [30]:
print(len(win_list))
print(len(lose_list))

307
79


In [ ]:
len({})